## Human Resources Data Analysis (Python, MySQL, PowerBI)

### Importing CSV to PostgreSQL databse

In [1]:
import psycopg2

In [2]:
try:
    connection = psycopg2.connect(
        user = "postgres",
#        password = "your_password",
        host = "localhost", # or the IP address of your PostgreSQL server
        port = "5432", # default PostgreSQL port
        database = "HR_Project"
    )

    cursor_post = connection.cursor()

    # Print PostgreSQL version
    cursor_post.execute("SELECT version();")
    record = cursor_post.fetchone()
    print("You are connected to - ", record)

except (Exception, psycopg2.Error) as error:
    print("Error while connecting to PostgreSQL", error)

# finally:
#     # Close the cursor and connection to avoid memory leaks
#     if (connection):
#         cursor_post.close()
#         connection.close()
#         print("PostgreSQL connection is closed")

You are connected to -  ('PostgreSQL 14.8 on aarch64-apple-darwin20.6.0, compiled by Apple clang version 12.0.5 (clang-1205.0.22.9), 64-bit',)


In [3]:
# drop tables with the same table name
try:
    # Roll back any previous transaction that ended in error
    connection.rollback()
    
    # Now you can perform other operations
    cursor_post.execute('DROP TABLE IF EXISTS human_resources;') 

except Exception as e:
    print("An error occurred:", e)

In [4]:
# execute the CREATE TABLE statement
cursor_post.execute("""
    CREATE TABLE IF NOT EXISTS Human_Resources (
        id VARCHAR(255), 
        first_name VARCHAR(255), 
        last_name VARCHAR(255), 
        birthdate VARCHAR(255), 
        gender VARCHAR(255), 
        race VARCHAR(255), 
        department VARCHAR(255), 
        jobtitle VARCHAR(255), 
        location VARCHAR(255), 
        hire_date VARCHAR(255), 
        termdate VARCHAR(255), 
        location_city VARCHAR(255), 
        location_state VARCHAR(255)
    );
""")
connection.commit()

In [5]:
table_name = "human_resources"

# Execute the SELECT statement
cursor_post.execute(f"""
    SELECT EXISTS (
        SELECT FROM information_schema.tables 
        WHERE  table_schema = 'public'
        AND    table_name   = '{table_name}'
    );
""")

# Fetch the result
exists = cursor_post.fetchone()[0]

if exists:
    print(f"The table '{table_name}' exists.")
else:
    print(f"The table '{table_name}' does not exist.")

The table 'human_resources' exists.


In [6]:
#open the csv file, save it as an object
my_file = open('Human_Resources.csv')
print('file opened in memory')

file opened in memory


In [7]:
# upload to db
SQL_STATEMENT = '''
    COPY Human_Resources 
    FROM STDIN 
    WITH
        CSV
        HEADER
        DELIMITER AS  ','
'''

In [8]:
try:
    connection.rollback()  # Roll back any previous failed transaction
    cursor_post.copy_expert(sql=SQL_STATEMENT, file=my_file)
    print("File copied to db")
except Exception as e:
    print("An error has occurred:", e)
    connection.rollback()  # Roll back this transaction if an error occurred

File copied to db


In [9]:
# optional:
cursor_post.execute("grant select on table Human_Resources to public")
connection.commit()

# close the connection
cursor_post.close()

In [10]:
# Your SQL query
sql_query = "SELECT * FROM Human_Resources;"

# Execute the SQL query
cursor_post.execute(sql_query)

# Fetch all the rows
rows = cursor_post.fetchall()
# Print the rows
for row in rows:
    print(row)